# Configure Scan & System Health Displays Test

## Table of Contents
#### [1 - Global Configuration](#1-global-configuration)
#### [2 - Setup Test Equipment](#2-test-equipment-setup)
#### [3 - Configure Scan Tests](#3-configure-scan-tests)
#### [4 - Debug Telescope](#4-debug-telescope)
#### [5 - Dashboard Links](#5-dashboard-links)
#### [6 - System Health Displays Tests](#6-system-health-displays-tests)

## 1 Global Configuration

### 1.1 Import dependencies

In [ ]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
from typing import List

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec, get_default_target_specs_sb
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

### 1.2 Setup Global Variables and Configuration

In [ ]:
debug_mode = True  # This setting enables printing of diagnostics
enable_logging = True  # This enables logging and sets the global log_level to debug
dishlmc_enabled = True  # Set this to true if you have a dish LMC deployment
executon_environment = Environment.Staging  # Choose option CI / Integration / Staging
branch_name = "none"  # Set this if you are using an on-demand deployment (i.e. Environment.CI)
if enable_logging:
    ska_ser_logging.configure_logging(logging.DEBUG)
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
# namespace_override parameter can be used to override auto-configured SUT namespace
sut_namespace_override = ""
subarray_count = 1
subarray_id = 1
sut = TangoSUTDeployment(
    branch_name,
    executon_environment,
    namespace_override=sut_namespace_override,
    subarray_index=subarray_id,
)
print(f"SUT configured: {str(sut)}")
dish_ids = ["001", "036"]
# namespace_override parameter can be used to override auto-configured dish namespace
dish_namespace_overrides = ["", ""]
dish_deployments: List[TangoDishDeployment] = []
if dishlmc_enabled:
    for i, d in enumerate(dish_ids):
        dish = TangoDishDeployment(
            f"ska{d}",
            branch_name=branch_name,
            environment=executon_environment,
            namespace_override=dish_namespace_overrides[i],
        )
        print(f"Dish {d} configured: {dish}")
        dish_deployments.append(dish)

timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir, exist_ok=True)
# we disable qa as it is not been properly verified
disable_qa()

### 1.3 Test Connections to Namespaces

In [ ]:
print(os.environ["TANGO_HOST"])

In [ ]:
sut.smoke_test()
test_equipment.smoke_test()
for dish_deployment in dish_deployments:
    dish_deployment.smoke_test()

### 1.4 Export System Configuration

In [ ]:
deployment: TangoDeployment
for deployment in [sut, test_equipment, *dish_deployments]:
    deployment.export_chart_configuration(output_dir=notebook_output_dir)

## 2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 2.1 Configure Test Equipment State 

In [ ]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

### 2.2 Print Test Equipment Diagnostics

In [ ]:
test_equipment.print_diagnostics()

### 2.3 Create Test Equipment Plot

In [ ]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

### 2.4 Turn offline Test Equipment devices ONLINE

In [ ]:
# set any offline devices to online
test_equipment.turn_online()

### 2.5 Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

### 2.6 Configure Signal Generator and set noise

In [ ]:
frequency_to_set = 800e6

signal_generator = test_equipment.signal_generator
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute("frequency", frequency_to_set)
time.sleep(1)
print(f"Updated signal generator frequency: {signal_generator.frequency}")
assert (
    signal_generator.frequency == frequency_to_set
), f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"

## 3 Configure Scan Tests

Use OSO scripting interface to TMC to run the script for assigning resources for a single sub-array, configuring and running a  scan.  

### 3.1 Setup Telescope Monitoring

#### 3.1.1 Configure Telescope Monitoring

In [ ]:
# setup monitoring
# use telescope state object for state monitoring
device_model = TelescopeDeviceModel(dish_ids, subarray_count)
telescope_state = get_telescope_state(device_model, sut)
# use monitor plot as a dashboard
telescope_monitor_plot = TelescopeMononitorPlot(plot_width=900, plot_height=200)
# set up events to monitor
telescope_state.subscribe_to_on_off(telescope_monitor_plot.observe_telescope_on_off)
telescope_state.subscribe_to_subarray_resource_state(
    telescope_monitor_plot.observe_subarray_resources_state
)
telescope_state.subscribe_to_subarray_configurational_state(
    telescope_monitor_plot.observe_subarray_configuration_state
)
telescope_state.subscribe_to_subarray_scanning_state(
    telescope_monitor_plot.observe_subarray_scanning_state
)
output_notebook()

#### 3.1.2 Open the inline dashboard
Start the simple inline dashboard showing current state of the Telescope and resource assignment and configuration status.

In [ ]:
telescope_monitor_plot.show()
telescope_state.activate()
telescope_state.wait_til_ready(2)

### 3.2 Print System Diagnostics

#### 3.2.1 Print TMC Diagnostics

In [ ]:
sut.print_tmc_diagnostics()

#### 3.2.2 Print CSP-LMC Diagnostics

In [ ]:
sut.print_csp_diagnostics()

#### 3.2.3 Print CBF Diagnostics

In [ ]:
sut.print_cbf_diagnostics()

#### 3.2.4 Print SDP Diagnostics

In [ ]:
sut.print_sdp_diagnostics()

#### 3.2.5 Print Dish-LMC Diagnostics

In [ ]:
for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id} - {dish_deployment.namespace}: Diagnostics")

    dish_deployment.print_diagnostics()

#### 3.2.6 Print Full System Diagnostics

In [ ]:
print("SUT: Full Diagnostics")
sut.print_full_diagnostics()

for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id}: Full Diagnostics")
    dish_deployment.print_full_diagnostics()

print("Test Equipment: Full Diagnostics")
test_equipment.print_full_diagnostics()

### 3.3 Setup ODA

In [ ]:
os.environ["ODA_URI"] = (
    "http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/"
)
eb_id = oda_helper.create_eb()
print(f"Execution Block ID: {eb_id}")

### 3.4 Initialise Telescope and Subarray
Create Subarray and Telescope instances.

In [ ]:
sub = SubArray(subarray_id)
tel = Telescope()

### 3.5 Load VCC Configuration in TMC

In [ ]:
# This should only be executed for a fresh deployment (i.e. Telescope is OFF.
# If you have restarted the subarray, you should not run this command
sut.load_dish_vcc_config()

### 3.6 Turn telescope ON

In [ ]:
# set to ON only if OFF
# If you have restarted the subarray, you should not run this command (Telescope is already ON)
# dish_lmc mode must be in LP_standby and before trying to turn the telescope ON
# Takes about 1m20s
if telescope_monitor_plot.on_off_state == "OFF":  # e.g. purple
    tel.on()
else:
    assert (
        telescope_monitor_plot.on_off_state == "ON"
    ), f"Cant continue with telescope in {telescope_monitor_plot.on_off_state}"

### 3.7 Observation Definition

#### 3.7.1 Create the high level observation specifications in terms of target specs

Note :- Users may currently modify the values by replacing the example values as given for each field within Target specification section.

In [ ]:
dish_ids = [d.dish_id.upper() for d in dish_deployments]
default_target_specs = get_default_target_specs_sb(dish_ids)
observation = ObservationSB(target_specs=default_target_specs)

target_specs = {
    "flux calibrator": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="flux calibrator",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels9",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
    "M87": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="M87",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels10",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
}


channel_configuration = [
    Channel(
        spectral_window_id="fsp_1_channels",
        count=14880,
        start=0,
        stride=2,
        freq_min=0.35e9,
        freq_max=0.368e9,
        link_map=[[0, 0], [200, 1], [744, 2], [944, 3]],
    )
]

for key, value in target_specs.items():
    observation.add_channel_configuration(value.channelisation, channel_configuration)

observation.add_target_specs(target_specs)

for target_id, target in target_specs.items():
    observation.add_scan_type_configuration(
        config_name=target_id,
        beams={"vis0": BeamMapping(beam_id="vis0", field_id="M83")},
        derive_from=".default",
    )
scan_def_id = "flux calibrator"
observation.add_scan_sequence([scan_def_id])

#### 3.7.2 Mid configuration schema input used by observing commands

[Configuration Schemas-OET→TMC(Mid)](https://developer.skao.int/projects/ska-telmodel/en/latest/)

In [ ]:
telescope_monitor_plot.show()

#### 3.7.3 Create Scheduling Block Definition(SBD) Instance and save it into the ODA

In [ ]:
observation.eb_id = eb_id
pdm_allocation = observation.generate_pdm_object_for_sbd_save(target_specs)

sbd = oda_helper.save(pdm_allocation)
sbd_id = sbd.sbd_id
pdm_allocation.sbd_id = sbd_id
print(f"Saved Scheduling Block Definition Instance in ODA: SBD_ID={sbd_id}")

### 3.8 Assign Resources
Assign the requested resources to a Subarray

In [ ]:
assign_request = observation.generate_allocate_config_sb(pdm_allocation).as_object

if debug_mode:
    request_json = get_request_json(assign_request, AssignResourcesRequest, True)
    print("AssignResourcesRequest:", json.dumps(json.loads(request_json), indent=2))

sub.assign_from_cdm(assign_request, timeout=120)

In [ ]:
telescope_monitor_plot.show()

# :warning: Temporary TMC workaround :warning:
At the end of PI22, DishLMC 3.3.0 was released with an updated behaviour that was not yet catered for by the latest version of TMC. In order to compensate for this, we temporarily implement this section (taken from the [DishLMC testing suite](https://gitlab.com/ska-telescope/ska-mid-dish-manager/-/blob/main/tests/acceptance/test_track_cmds.py?ref_type=heads#L91-112)).

:warning: This should be removed as soon as a new TMC version is made available. More details in [this Slack thread](https://skao.slack.com/archives/C0625UVDVC6/p1716820238256029?thread_ts=1716804215.873139&cid=C0625UVDVC6).

In [ ]:
from astropy.time import Time


def get_tai_from_unix_s(unix_s: float) -> float:
    """
    Calculate atomic time in seconds from unix time in seconds.

    :param unix_s: Unix time in seconds

    :return: atomic time (tai) in seconds
    """
    astropy_time_utc = Time(unix_s, format="unix")
    return astropy_time_utc.unix_tai


for dish in dish_deployments:
    dish_manager_proxy = dish.dish_manager
    current_pointing = dish_manager_proxy.achievedPointing
    current_az = current_pointing[1]
    current_el = current_pointing[2]

    current_time_tai_s = get_tai_from_unix_s(time.time())

    # Directions to move values
    az_dir = 1 if current_az < 350 else -1
    el_dir = 1 if current_el < 80 else -1

    track_table = [
        current_time_tai_s + 3,
        current_az + 1 * az_dir,
        current_el + 1 * el_dir,
        current_time_tai_s + 5,
        current_az + 2 * az_dir,
        current_el + 2 * el_dir,
        current_time_tai_s + 7,
        current_az + 3 * az_dir,
        current_el + 3 * el_dir,
        current_time_tai_s + 9,
        current_az + 4 * az_dir,
        current_el + 4 * el_dir,
        current_time_tai_s + 11,
        current_az + 5 * az_dir,
        current_el + 5 * el_dir,
    ]

    dish_manager_proxy.programTrackTable = track_table

### 3.9 Configure Scan
Configure the telescope  on first target in sequence - may be modified to configure and run multiple targets at a later time.

In [ ]:
configure_object = observation.generate_scan_config_sb(
    pdm_observation_request=pdm_allocation,
    scan_definition_id=scan_def_id,
    scan_duration=10.0,
).as_object

if debug_mode:
    cfg_json = get_request_json(configure_object, ConfigureRequest)
    print(f"ConfigureRequest={cfg_json}")

sub.configure_from_cdm(configure_object, timeout=120)
time.sleep(2)

In [ ]:
telescope_monitor_plot.show()

### 3.10 Run the Scan

In [ ]:
sub.scan(timeout=120)
telescope_monitor_plot.show()

### 3.11 Post Observation teardown
If the observation executed successfully, you can use the following commands to reset the telescope.

#### 3.11.1 Clear scan configuration 

In [ ]:
sub.end()
telescope_monitor_plot.show()

#### 3.11.2 Release Subarray resources

In [ ]:
sub.release()
telescope_monitor_plot.show()

### 3.12 Reset the Telescope (On Failure)
Set booleans to True to reset the system after a failed execution.

#### 3.12.1 Reset the Subarray

In [ ]:
sub.abort()
time.sleep(3)
sub.restart()

#### 3.12.2 Reset the Dish

In [ ]:
dish_deployment: TangoDishDeployment
for dish_deployment in dish_deployments:
    dish_deployment.reset_dish()

## 4 Debug Telescope

### 4.1 Display Telescope State

In [ ]:
telescope_state.state

### 4.2 Display Telescope State Subscriptions

In [ ]:
telescope_state.state_monitor.subscriptions

### 4.3 Display Dish LMC State

In [ ]:
dishes_to_debug = [d.dish_id for d in dish_deployments]
for dish in dish_deployments:
    if dish.dish_id in dishes_to_debug:
        dish.print_diagnostics()

### 4.4 Display CSP LMC State

In [ ]:
sut.print_csp_diagnostics()

### 4.5 Display TMC State

In [ ]:
sut.print_tmc_diagnostics()

### 4.6 Display SDP State

In [ ]:
sut.print_sdp_diagnostics()

## 5 Dashboard Links

### 5.1 Taranta

In [ ]:
print(f"SUT: {sut.taranta_endpoint}")
print(f"Test Equipment: {test_equipment.taranta_endpoint}")
for dish in dish_deployments:
    print(f"Dish LMC {dish.dish_id}: {dish.taranta_endpoint}")

### 5.2 Data Product Dashboard

https://k8s.miditf.internal.skao.int/ska-dpd/dashboard/

### 5.3 SDP Signal Displays

In [ ]:
print(sut.signal_displays_endpoint)

## 6 System Health Displays Tests

### Copied from Jama 2024-03-28

### 6.1 Display the system status and bring the telescope into a known operating state of Standby (SPFC, Dish Structure Controller Simulator, SPFRx, CBF, SDP)

Running steps 0.1 and 2.1-2.6 should get the telescope into a standby state which can be checked by running 3.1 and 3.3.1

### 6.2 Use TMC and LMC dashboards to show status and sensor values from each subsystem. Temperature sensors should be displayed where available.
Run dashboards with links in step 4.2

### 6.3 Use the EDA configuration supplied by each product to configure the EDA in the ITF to subscribe to a range of relevant Tango attribute sensors from each product. Temperature sensors should be displayed where available.

### 6.4 Show that archived sensor values can be displayed or plotted over a configurable time period

### 6.5 Show that several historical sensor values can be plotted on the same axis.

### 6.6 Configure the alarm handler in the ITF to subscribe to a range of relevant Tango attribute sensors from each product.

### 6.7 Use alarm handler to display which sensors are currently in Warning and Alarm status.

### 6.8 Show the drill-down display from the top level system to the products in the system, showing sensor values and associated health and alarm displays

### 6.9 Force an alarm condition and verify that the alarm is reported with the correct timestamp by the alarm handler.  Repeat for various sensors. (it may be necessary to change the alarm threshold values to create the alarm condition)

### 6.10 Perform network interruption conditions (eg. remove a network cable for a period) to demonstrate communication loss alarm indications and recovery.

### 6.11 Display simulated pointing position of antennas and target information